In [1]:
import requests
import pandas as pd
import os
import pandas_ta as ta
import matplotlib.pyplot as plt; plt.style.use('seaborn')

os.chdir('/Users/ashleyyan/Downloads/stock-project')

key = 'JJi3jpQFqO4peWflZDIK2nNaVsq1EQE4'

/var/folders/v5/jb3ydpjj5bxdqk7b8fnr0p7r0000gn/T/ipykernel_19464/1304903420.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  import matplotlib.pyplot as plt; plt.style.use('seaborn')


In [2]:
def get_stocks(ticker, start, end):
    url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{start}/{end}?adjusted=true&sort=asc&limit=50000&apiKey={key}'
    json = requests.get(url).json()
    return json

In [13]:
response = get_stocks('I:SPX', '2023-03-13', '2023-08-07')
data = response['results']
data

[{'o': 3835.12, 'c': 3824.94, 'h': 3835.12, 'l': 3819.07, 't': 1678714200000},
 {'o': 3824.94,
  'c': 3822.9700000000003,
  'h': 3825.71,
  'l': 3818.63,
  't': 1678714260000},
 {'o': 3823.55, 'c': 3820.6, 'h': 3825.61, 'l': 3818.05, 't': 1678714320000},
 {'o': 3821.31,
  'c': 3826.3,
  'h': 3827.9500000000003,
  'l': 3817.67,
  't': 1678714380000},
 {'o': 3825.87,
  'c': 3819.9900000000002,
  'h': 3825.87,
  'l': 3819.54,
  't': 1678714440000},
 {'o': 3819.6, 'c': 3813.78, 'h': 3819.6, 'l': 3813.78, 't': 1678714500000},
 {'o': 3814.08, 'c': 3818.3, 'h': 3819.02, 'l': 3814.08, 't': 1678714560000},
 {'o': 3817.57, 'c': 3813.52, 'h': 3817.57, 'l': 3813.1, 't': 1678714620000},
 {'o': 3813.42, 'c': 3810.05, 'h': 3813.42, 'l': 3808.88, 't': 1678714680000},
 {'o': 3810.2400000000002,
  'c': 3815.15,
  'h': 3815.15,
  'l': 3808.86,
  't': 1678714740000},
 {'o': 3815.9300000000003,
  'c': 3816.1,
  'h': 3819.53,
  'l': 3815.4,
  't': 1678714800000},
 {'o': 3816.98, 'c': 3812.14, 'h': 3816.98, 

In [14]:
def convert_to_csv(data):
    time_col = []
    open_col = []
    high_col = []
    low_col = []
    close_col = []
    for i in data:
        time_col.append(i['t'])
        open_col.append(i['o'])
        high_col.append(i['h'])
        low_col.append(i['l'])
        close_col.append(i['c'])
    dictionary = {'time': time_col, 'open': open_col, 'high': high_col, 'low': low_col, 'close': close_col}
    df = pd.DataFrame(dictionary)
    return df

In [21]:
df = convert_to_csv(data)
df

,time,open,high,low,close
0,1678714200000,3835.12,3835.12,3819.07,3824.94
1,1678714260000,3824.94,3825.71,3818.63,3822.97
2,1678714320000,3823.55,3825.61,3818.05,3820.60
3,1678714380000,3821.31,3827.95,3817.67,3826.30
4,1678714440000,3825.87,3825.87,3819.54,3819.99
...,...,...,...,...,...
41607,1691439420000,4518.44,4518.44,4518.44,4518.44
41608,1691439480000,4518.44,4518.44,4518.44,4518.44
41609,1691439540000,4518.44,4518.44,4518.44,4518.44
41610,1691439600000,4518.44,4518.44,4518.44,4518.44


In [22]:
def get_mins(num_days):
    mins_per_day = len(df)/180
    return mins_per_day*num_days

In [23]:
df['RSI'] = ta.rsi(df['close'], length=get_mins(14)) #14 day
df['EMAF'] = ta.ema(df['close'], length=get_mins(12)) #12 day
df['EMAM'] = ta.ema(df['close'], length=get_mins(26)) #26 day
df.ta.macd(close=df['close'], fast=get_mins(12), slow=get_mins(26), signal=get_mins(9), append=True)
df.ta.bbands(close=df['close'], length=get_mins(20), append=True)
df.ta.atr(high=df['high'], low=df['low'], close=df['close'], window=get_mins(14), append=True)
df

,time,open,high,low,close,RSI,EMAF,EMAM,MACD_2774_6010_2080,MACDh_2774_6010_2080,MACDs_2774_6010_2080,BBL_4623_2.0,BBM_4623_2.0,BBU_4623_2.0,BBB_4623_2.0,BBP_4623_2.0,ATRr_14
0,1678714200000,3835.12,3835.12,3819.07,3824.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1678714260000,3824.94,3825.71,3818.63,3822.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1678714320000,3823.55,3825.61,3818.05,3820.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1678714380000,3821.31,3827.95,3817.67,3826.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1678714440000,3825.87,3825.87,3819.54,3819.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41607,1691439420000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.246090,4527.658554,0.587536,-13.437894,14.025431,4486.316930,4549.419178,4612.521426,2.774079,0.254532,0.343525
41608,1691439480000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.239023,4527.655487,0.583536,-13.428976,14.012512,4486.303410,4549.412226,4612.521042,2.774372,0.254613,0.318987
41609,1691439540000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.231960,4527.652420,0.579540,-13.420062,13.999602,4486.289843,4549.405220,4612.520596,2.774665,0.254694,0.296203
41610,1691439600000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.224903,4527.649355,0.575548,-13.411152,13.986700,4486.276326,4549.398263,4612.520200,2.774958,0.254774,0.275045


In [24]:
df = df.dropna()
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [26]:
df = df.rename(columns = {'MACD_2774_6010_2080': 'MACD', 'MACDh_2774_6010_2080': 'MACD_histogram', 
                          'MACDs_2774_6010_2080': 'MACD_signal', 'BBL_4623_2.0': 'BBL', 'BBM_4623_2.0': 'BBM', 
                          'BBU_4623_2.0': 'BBU', 'BBB_4623_2.0': 'BBB', 'BBP_4623_2.0':'BBP', 'ATRr_14':'ATR'})
df

,time,open,high,low,close,RSI,EMAF,EMAM,MACD,MACD_histogram,MACD_signal,BBL,BBM,BBU,BBB,BBP,ATR
0,1681149000000,4100.50,4100.75,4100.00,4100.58,51.314859,4074.699538,4027.271355,47.428183,-5.601752,53.029935,3920.884614,4041.729230,4162.573846,5.979847,0.743498,0.922885
1,1681149060000,4100.64,4100.72,4099.96,4100.08,51.306065,4074.717830,4027.295580,47.422250,-5.602296,53.024546,3920.906905,4041.753435,4162.599965,5.979906,0.741325,0.911251
2,1681149120000,4100.12,4100.80,4100.12,4100.38,51.311073,4074.736325,4027.319897,47.416428,-5.602727,53.019156,3920.929868,4041.777960,4162.626052,5.979947,0.742462,0.897590
3,1681149180000,4100.42,4100.78,4099.35,4099.46,51.294889,4074.754144,4027.343900,47.410245,-5.603521,53.013765,3920.951719,4041.801813,4162.651906,5.980011,0.738553,0.935619
4,1681149240000,4099.45,4099.70,4098.24,4098.32,51.274843,4074.771129,4027.367515,47.403614,-5.604760,53.008373,3920.973465,4041.825293,4162.677121,5.980062,0.733735,0.973075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33519,1691439420000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.246090,4527.658554,0.587536,-13.437894,14.025431,4486.316930,4549.419178,4612.521426,2.774079,0.254532,0.343525
33520,1691439480000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.239023,4527.655487,0.583536,-13.428976,14.012512,4486.303410,4549.412226,4612.521042,2.774372,0.254613,0.318987
33521,1691439540000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.231960,4527.652420,0.579540,-13.420062,13.999602,4486.289843,4549.405220,4612.520596,2.774665,0.254694,0.296203
33522,1691439600000,4518.44,4518.44,4518.44,4518.44,49.867417,4528.224903,4527.649355,0.575548,-13.411152,13.986700,4486.276326,4549.398263,4612.520200,2.774958,0.254774,0.275045


In [27]:
df.to_csv('s&p_data')